In [1]:
import params
import dataset

import cv2
import numpy as np

import tqdm
from tqdm import tqdm_notebook as tqdm

import json

import matplotlib.pyplot as plt

In [2]:
def read_image_and_mask(img_name, mask_name):
    img = cv2.imread(img_name, cv2.IMREAD_COLOR)
    mask = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)
    
    img = cv2.resize(img, (params.GetImageSize(), params.GetImageSize()))
    mask = cv2.resize(mask, (params.GetImageSize(), params.GetImageSize()))  
    
#     height, width = mask.shape
#     class_masks = np.zeros((height, width, params.GetClassesCount()))
#     for i in range(class_masks.shape[2]):
#         class_masks[:, :, i] = (mask == (i + 1))
    
    return img, mask

In [3]:
average_image = np.zeros((params.GetImageSize(), params.GetImageSize(), 3)).astype(np.float32)
classes_counts = np.zeros((params.GetClassesCount())).astype(np.float32)

In [4]:
files, masks = dataset.read_data_set(params.validation_data_path)

all_files = zip(files, masks)

validation_files_cnt = len(all_files)

for ind in tqdm(range(validation_files_cnt)):
    img_name, mask_name = all_files[ind]
    img, mask = read_image_and_mask(img_name, mask_name)
    
    average_image = average_image + img
    for i in range(params.GetClassesCount()):
        classes_counts[i] = classes_counts[i] + np.sum(mask == (i + 1))

In [5]:
files, masks = dataset.read_data_set(params.training_data_path)

all_files = zip(files, masks)

training_files_cnt = len(all_files)

for ind in tqdm(range(training_files_cnt)):
    img_name, mask_name = all_files[ind]
    img, mask = read_image_and_mask(img_name, mask_name)
    
    average_image = average_image + img
    for i in range(params.GetClassesCount()):
        classes_counts[i] = classes_counts[i] + np.sum(mask == (i + 1))

In [6]:
average_image = (average_image / float(training_files_cnt + validation_files_cnt)).astype(np.uint8)
classes_counts_sum = np.sum(classes_counts)
classes_weights = 1.0 - classes_counts / classes_counts_sum

In [7]:
classes_counts, classes_weights, classes_counts_sum, classes_counts / classes_counts_sum

(array([1.033189e+08, 6.115082e+08, 6.731249e+07, 4.272151e+06],
       dtype=float32),
 array([0.86861986, 0.2224071 , 0.9144055 , 0.9945675 ], dtype=float32),
 786411800.0,
 array([0.13138016, 0.7775929 , 0.08559445, 0.00543246], dtype=float32))

In [8]:
cv2.imwrite(params.average_image, average_image)

True

In [9]:
data_stats = {
    "classes_weights": classes_weights.tolist()
}
data_file = open(params.data_info_file, "w")
# magic happens here to make it pretty-printed
json.dump(data_stats, data_file, indent=4, sort_keys=True)
data_file.close()